In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import splitfolders


In [9]:
class SkinAnomalyDetector:
    def __init__(self, 
                 dataset_path='Split_smol', 
                 img_height=224, 
                 img_width=224, 
                 batch_size=32):
        """
        Initialize Skin Anomaly Detection System

        Args:
            dataset_path (str): Path to skin condition dataset
            img_height (int): Image height for preprocessing
            img_width (int): Image width for preprocessing
            batch_size (int): Training batch size
        """
        self.dataset_path = dataset_path
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size

        # Prepare data generators
        self.train_generator = None
        self.validation_generator = None
        self.prepare_data_generators()

        # Model 
        self.model = None

    def prepare_data_generators(self):
        """
        Prepare data generators with augmentation, handling missing files gracefully
        """
        try:
            # Training data augmentation
            train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )

            # Validation data only rescaling
            validation_datagen = ImageDataGenerator(rescale=1./255)

            # Generate data generators
            self.train_generator = train_datagen.flow_from_directory(
                f'{self.dataset_path}/train',
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical'
            )

            self.validation_generator = validation_datagen.flow_from_directory(
                f'{self.dataset_path}/val',
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical'
            )

        except FileNotFoundError as e:
            print(f"Error: {e}")
            print("Please ensure the dataset is organized correctly.")

    def build_model(self):
        """
        Build Convolutional Neural Network for Skin Anomaly Detection
        """
        self.model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.img_height, self.img_width, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),

            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),

            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),

            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),

            layers.Dense(len(self.train_generator.class_indices), activation='softmax')
        ])

        self.model.compile(
            optimizer=optimizers.Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model.summary()

    def train_model(self, epochs=50):
        if not self.model:
            self.build_model()

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint('best_skin_anomaly_model.h5', save_best_only=True, monitor='val_accuracy')

        try:
            history = self.model.fit(
                self.train_generator,
                validation_data=self.validation_generator,
                epochs=epochs,
                callbacks=[early_stopping, model_checkpoint]
            )

            return history

        except FileNotFoundError as e:
            print(f"Training error: {e}")
            print("Check your dataset path and structure.")

    def evaluate_model(self):
        test_datagen = ImageDataGenerator(rescale=1./255)
        try:
            test_generator = test_datagen.flow_from_directory(
                f'{self.dataset_path}/val',
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical'
            )

            test_loss, test_accuracy = self.model.evaluate(test_generator)
            print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

        except FileNotFoundError as e:
            print(f"Evaluation error: {e}")
            print("Check your dataset path and structure.")

    def predict_image(self, image_path):
        try:
            img = tf.keras.preprocessing.image.load_img(
                image_path,
                target_size=(self.img_height, self.img_width)
            )
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0) / 255.0

            predictions = self.model.predict(img_array)
            class_names = list(self.train_generator.class_indices.keys())

            predicted_class = class_names[np.argmax(predictions[0])]
            confidence = np.max(predictions[0])

            return predicted_class, confidence

        except FileNotFoundError as e:
            print(f"Prediction error: {e}")
            print("Check your image path.")

if __name__ == "__main__":
    detector = SkinAnomalyDetector()
    detector.train_model(epochs=20)
    detector.evaluate_model()


Found 697 images belonging to 9 classes.
Found 181 images belonging to 9 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 111, 111, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 54, 54, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 26, 26, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,279,625 (84.99 MB)

 Trainable params: 22,279,177 (84.99 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/20
 2/22 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.0938 - loss: 4.0400 

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: [Errno 2] No such file or directory: 'Split_smol/train\\Tinea Ringworm Candidiasis\\aug_0_natural-treatment-Daad-khaj-khujli-ringworm-home-treatment-home-remedies-fungal-infection-treatment-symptom-pictures-diagnosis-hindi-gimtrend.blogspot.com.jpeg'
Traceback (most recent call last):

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    ret = func(*args)

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\

In [12]:
class SkinAnomalyDetector:
    def __init__(self, 
                 dataset_path='Split_smol', 
                 img_height=224, 
                 img_width=224, 
                 batch_size=32):
        """
        Initialize Skin Anomaly Detection System

        Args:
            dataset_path (str): Path to skin condition dataset
            img_height (int): Image height for preprocessing
            img_width (int): Image width for preprocessing
            batch_size (int): Training batch size
        """
        self.dataset_path = dataset_path
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size

        # Prepare data generators
        self.train_generator = None
        self.validation_generator = None
        self.class_indices = None
        self.prepare_data_generators()

        # Model 
        self.model = None

    def prepare_data_generators(self):
        """
        Prepare data generators with augmentation, handling missing files gracefully
        """
        try:
            # Training data augmentation
            train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )

            # Validation data only rescaling
            validation_datagen = ImageDataGenerator(rescale=1./255)

            # Original generators
            train_gen = train_datagen.flow_from_directory(
                f'{self.dataset_path}/train',
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical'
            )

            val_gen = validation_datagen.flow_from_directory(
                f'{self.dataset_path}/val',
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical'
            )

            # Store class indices and set generators
            self.class_indices = train_gen.class_indices
            self.train_generator = lambda: train_gen
            self.validation_generator = lambda: val_gen

        except FileNotFoundError as e:
            print(f"Error: {e}")
            print("Please ensure the dataset is organized correctly.")

    def build_model(self):
        """
        Build Convolutional Neural Network for Skin Anomaly Detection
        """
        self.model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.img_height, self.img_width, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),

            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),

            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),

            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),

            layers.Dense(len(self.class_indices), activation='softmax')
        ])

        self.model.compile(
            optimizer=optimizers.Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model.summary()

    def train_model(self, epochs=50):
        if not self.model:
            self.build_model()

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint('best_skin_anomaly_model.h5', save_best_only=True, monitor='val_accuracy')

        try:
            history = self.model.fit(
                self.train_generator(),
                validation_data=self.validation_generator(),
                epochs=epochs,
                callbacks=[early_stopping, model_checkpoint]
            )

            return history

        except FileNotFoundError as e:
            print(f"Training error: {e}")
            print("Check your dataset path and structure.")

    def evaluate_model(self):
        test_datagen = ImageDataGenerator(rescale=1./255)
        try:
            test_generator = test_datagen.flow_from_directory(
                f'{self.dataset_path}/val',
                target_size=(self.img_height, self.img_width),
                batch_size=self.batch_size,
                class_mode='categorical'
            )

            test_loss, test_accuracy = self.model.evaluate(test_generator)
            print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

        except FileNotFoundError as e:
            print(f"Evaluation error: {e}")
            print("Check your dataset path and structure.")

    def predict_image(self, image_path):
        try:
            img = tf.keras.preprocessing.image.load_img(
                image_path,
                target_size=(self.img_height, self.img_width)
            )
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0) / 255.0

            predictions = self.model.predict(img_array)
            class_names = list(self.class_indices.keys())

            predicted_class = class_names[np.argmax(predictions[0])]
            confidence = np.max(predictions[0])

            return predicted_class, confidence

        except FileNotFoundError as e:
            print(f"Prediction error: {e}")
            print("Check your image path.")

if __name__ == "__main__":
    detector = SkinAnomalyDetector()
    detector.train_model(epochs=20)
    detector.evaluate_model()

Found 697 images belonging to 9 classes.
Found 181 images belonging to 9 classes.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 111, 111, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 54, 54, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 26, 26, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,279,625 (84.99 MB)

 Trainable params: 22,279,177 (84.99 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/20
 8/22 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.1904 - loss: 4.0719

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: [Errno 2] No such file or directory: 'Split_smol/train\\Tinea Ringworm Candidiasis\\aug_0_natural-treatment-Daad-khaj-khujli-ringworm-home-treatment-home-remedies-fungal-infection-treatment-symptom-pictures-diagnosis-hindi-gimtrend.blogspot.com.jpeg'
Traceback (most recent call last):

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    ret = func(*args)

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\91623\.jupyter\ujjwal phython\lib\

In [13]:
import os

def list_directories(path):
    for dirpath, dirnames, filenames in os.walk(path):
        print(f"Directory: {dirpath}")
        for dirname in dirnames:
            print(f"  Sub-directory: {os.path.join(dirpath, dirname)}")
        for filename in filenames:
            print(f"    File: {os.path.join(dirpath, filename)}")

list_directories('Split_smol')


Directory: Split_smol
  Sub-directory: Split_smol\train
  Sub-directory: Split_smol\val
Directory: Split_smol\train
  Sub-directory: Split_smol\train\Actinic keratosis
  Sub-directory: Split_smol\train\Atopic Dermatitis
  Sub-directory: Split_smol\train\Benign keratosis
  Sub-directory: Split_smol\train\Dermatofibroma
  Sub-directory: Split_smol\train\Melanocytic nevus
  Sub-directory: Split_smol\train\Melanoma
  Sub-directory: Split_smol\train\Squamous cell carcinoma
  Sub-directory: Split_smol\train\Tinea Ringworm Candidiasis
  Sub-directory: Split_smol\train\Vascular lesion
Directory: Split_smol\train\Actinic keratosis
    File: Split_smol\train\Actinic keratosis\ISIC_0024468.jpg
    File: Split_smol\train\Actinic keratosis\ISIC_0024470.jpg
    File: Split_smol\train\Actinic keratosis\ISIC_0024707.jpg
    File: Split_smol\train\Actinic keratosis\ISIC_0024763.jpg
    File: Split_smol\train\Actinic keratosis\ISIC_0024771.jpg
    File: Split_smol\train\Actinic keratosis\ISIC_0024800.jp

Found 697 images belonging to 9 classes.


Found 181 images belonging to 9 classes.
Found 697 images belonging to 9 classes.


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 111, 111, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 54, 54, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 26, 26, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,279,625 (84.99 MB)

...

 Non-trainable params: 448 (1.75 KB)

Training Progress:   0%|          | 0/20 [00:00<?, ?it/s]

  31067/Unknown 31780s 1s/step - accuracy: 0.7575 - loss: 0.8931

Training Progress:   0%|          | 0/20 [8:49:41<?, ?it/s]


KeyboardInterrupt: 

In [14]:
import os

def list_directories_only(path):
    for dirpath, dirnames, _ in os.walk(path):
        print(f"Directory: {dirpath}")
        for dirname in dirnames:
            print(f"  Sub-directory: {os.path.join(dirpath, dirname)}")

list_directories_only('Split_smol')


Directory: Split_smol
  Sub-directory: Split_smol\train
  Sub-directory: Split_smol\val
Directory: Split_smol\train
  Sub-directory: Split_smol\train\Actinic keratosis
  Sub-directory: Split_smol\train\Atopic Dermatitis
  Sub-directory: Split_smol\train\Benign keratosis
  Sub-directory: Split_smol\train\Dermatofibroma
  Sub-directory: Split_smol\train\Melanocytic nevus
  Sub-directory: Split_smol\train\Melanoma
  Sub-directory: Split_smol\train\Squamous cell carcinoma
  Sub-directory: Split_smol\train\Tinea Ringworm Candidiasis
  Sub-directory: Split_smol\train\Vascular lesion
Directory: Split_smol\train\Actinic keratosis
Directory: Split_smol\train\Atopic Dermatitis
Directory: Split_smol\train\Benign keratosis
Directory: Split_smol\train\Dermatofibroma
Directory: Split_smol\train\Melanocytic nevus
Directory: Split_smol\train\Melanoma
Directory: Split_smol\train\Squamous cell carcinoma
Directory: Split_smol\train\Tinea Ringworm Candidiasis
Directory: Split_smol\train\Vascular lesion
Di

In [4]:
! pip install split-folders


In [6]:
! pip install tensorflow --index-url https://pypi.org/simple


^C


In [3]:
! pip install splitfolders

ERROR: Could not find a version that satisfies the requirement splitfolders (from versions: none)
ERROR: No matching distribution found for splitfolders


In [1]:
! pip install tensorflow --index-url https://pypi.org/simple
